# Encontrando as transações fraudulentas das contas correntes

Identificação de fraude: intervalo de transação menor que dois minutos

## Importando bibliotecas que iremos utilizar

In [168]:
# bibliotecas do pyspark para maior parte ad análise
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list

# biblioteca datetime para ajudar na formatação da coluna datahora
from datetime import timedelta, datetime

# biblioteca pandas para auxílio no dataset
import pandas as pd

# criando uma sessão Spark
spark = SparkSession.builder.appName('selecionar_fraudes').getOrCreate()

In [169]:
# carregando o arquivo CSV de transações como um Spark DataFrame
df = spark.read.csv('dataset_transacoes.csv', header=True)

# mostra o resultado
df.show()

+----+----------+-------+--------------------+----+------+-------+----------+
|  Id|Cliente_Id|  Valor|            DataHora|Hora|Minuto|Segundo|       Dia|
+----+----------+-------+--------------------+----+------+-------+----------+
|3120|       533|  5.001|2021-01-28T23:46:...|  23|    46|     47|2021-01-28|
|3119|       533| 24.999|2021-01-28T23:46:...|  23|    46|     47|2021-01-28|
|3108|       533|  5.001|2021-01-28T13:47:...|  13|    47|     37|2021-01-28|
|3107|       533|12.4995|2021-01-28T13:47:...|  13|    47|     36|2021-01-28|
|3106|       533|12.4995|2021-01-28T13:47:...|  13|    47|     36|2021-01-28|
|3092|       533| 24.999|2021-01-28T13:02:...|  13|     2|     57|2021-01-28|
|3091|       533|  5.001|2021-01-28T13:02:...|  13|     2|     57|2021-01-28|
|3079|       533| 24.999|2021-01-23T12:44:...|  12|    44|     31|2021-01-23|
|3078|       533|  5.001|2021-01-23T12:44:...|  12|    44|     30|2021-01-23|
|3069|       574|   20.0|2021-01-23T00:29:...|   0|    29|     5

## Fraudes out

In [170]:
# filtra as linhas com valor negativo na coluna "valor"
df_out = df.filter(col("Valor") < 0)

# mostra o resultado
df_out.show()

+----+----------+------+--------------------+----+------+-------+----------+
|  Id|Cliente_Id| Valor|            DataHora|Hora|Minuto|Segundo|       Dia|
+----+----------+------+--------------------+----+------+-------+----------+
|8607|       910|  -2.0|2022-01-19T20:15:...|  20|    15|     26|2022-01-19|
|8608|       910|  -2.0|2022-01-19T20:14:...|  20|    14|     56|2022-01-19|
|8609|       910|  -2.0|2022-01-19T20:14:...|  20|    14|     26|2022-01-19|
|8610|       910|  -2.0|2022-01-19T20:13:...|  20|    13|     56|2022-01-19|
|8612|       910|  -2.0|2022-01-19T20:13:...|  20|    13|     26|2022-01-19|
|8614|       910|  -2.0|2022-01-19T20:12:...|  20|    12|     56|2022-01-19|
|8573|       671| -10.0|2022-01-13T15:21:...|  15|    21|     25|2022-01-13|
|8574|       671| -10.0|2022-01-13T15:20:...|  15|    20|     55|2022-01-13|
|8575|       671|  -5.0|2022-01-13T15:20:...|  15|    20|     25|2022-01-13|
|8576|       671| -10.0|2022-01-13T15:19:...|  15|    19|     55|2022-01-13|

In [171]:
# ordenar por Cliente_Id, agrupar por Cliente_Id e coleta os valores de DataHora
df_agrupado = df_out.orderBy(['Cliente_Id', 'DataHora']).groupBy("Cliente_Id") \
                .agg(collect_list("DataHora").alias("DataHora"), \
                     collect_list("Id").alias("Id"), \
                     collect_list("Valor").alias("Valor"))
                
# mostra o resultado
df_agrupado.show()

+----------+--------------------+--------------------+--------------------+
|Cliente_Id|            DataHora|                  Id|               Valor|
+----------+--------------------+--------------------+--------------------+
|       718|[2021-07-29T20:05...|        [4825, 4826]|      [-20.0, -30.0]|
|       205|[2019-11-18T12:39...|[168, 169, 171, 1...|[-1.0, -5.0, -5.0...|
|       442|[2021-06-09T13:47...|              [3569]|              [-5.0]|
|       727|[2021-08-28T17:21...|  [5029, 5038, 5389]|[-20.0, -15.0, -1...|
|       495|[2020-10-31T15:39...|[1380, 1381, 1382...|[-1.0, -2.0, -2.0...|
|       685|[2021-06-09T23:42...|[3573, 3574, 3581...|[-2.0, -3.0, -5.0...|
|       387|[2021-06-09T21:21...|[3572, 3577, 3624...|[-20.0, -5.0, -20...|
|       724|[2021-08-20T00:20...|  [4935, 4936, 4951]|  [-2.0, -2.0, -5.0]|
|       534|[2020-12-19T02:13...|[2691, 2692, 2720...|[-1.0, -1.0, -2.0...|
|       586|[2021-02-25T21:16...|[3272, 3274, 3275...|[-10.0, -10.0, -2...|
|        73|

In [172]:
# itera sobre as linhas do DataFrame agrupado por cliente
lista = df_agrupado.collect()
for linha in lista:
    # recebe o valor do primeiro elemento da linha (cliente)
    cliente = linha[0]
    print(cliente)
    # itera simultaneamente sobre três listas (data, ID dos clientes e valor)
    for data, id, valor in zip(linha[1], linha[2], linha[3]):
        # imprime a data, o valor e o ID do cliente para cada elemento correspondente das três listas
        print(data, valor, id)

718
2021-07-29T20:05:00.000Z -20.0 4825
2021-07-29T20:34:58.000Z -30.0 4826
205
2019-11-18T12:39:53.000Z -1.0 168
2019-11-18T12:43:42.000Z -5.0 169
2019-11-18T12:48:37.000Z -5.0 171
2019-11-18T13:20:48.000Z -10.0 174
2019-11-18T21:39:33.000Z -1.0 176
2019-11-19T13:43:04.000Z -1.0 177
2019-11-19T13:43:19.000Z -1.0 178
2019-11-19T13:43:34.000Z -1.0 179
2019-11-19T13:51:54.000Z -1.0 180
2019-11-20T23:57:13.000Z -1.0 188
2019-11-22T16:03:55.000Z -1.0 190
2019-11-23T20:17:32.000Z -1.0 212
2019-11-24T22:34:21.000Z -5.0 213
2019-12-08T16:13:30.000Z -5.0 274
2019-12-08T16:19:04.000Z -5.0 275
2019-12-08T16:20:51.000Z -3.0 276
2019-12-08T16:23:32.000Z -5.0 277
2019-12-08T16:24:47.000Z -2.0 278
2019-12-08T17:08:47.000Z -10.0 279
2019-12-08T17:16:09.000Z -2.0 280
2019-12-08T20:59:20.000Z -5.0 283
2019-12-08T21:01:11.000Z -3.0 284
2019-12-23T12:03:18.000Z -20.0 349
2020-01-21T23:51:46.000Z -1.0 398
2020-01-21T23:53:23.000Z -1.0 399
2020-01-21T23:54:24.000Z -1.0 400
2020-01-21T23:55:40.000Z -1.0 401

In [173]:
# selecionando a coluna dataHora do dataset agrupado 
lista = df_agrupado.collect()

# definindo variável para 2 minutos
minute2 = timedelta(minutes=2)

# criando lista vazia chamada fraudes
fraudes = []

# estrutura de repetição para verificar em linha de cliente suas transações que possuem menos de dois minutos
# Percorre cada linha da lista
for linha in lista:
    # Extrai o primeiro elemento da linha, que é assumido como o identificador do cliente
    cliente = linha[0]
    # Imprime o cliente
    print('cliente:', cliente)
    # Compara a data da linha atual com a data da linha anterior e o ID do cliente da linha atual com o ID do cliente da linha anterior
    data_anterior = None
    id_anterior = None
    # verificamos cada dataHora da lista de transações do cliente e separamos as com diferença de menos de 2 min
    for data, id, valor in zip(linha[1], linha[2], linha[3]):
        data = datetime.strptime(data[:-5], '%Y-%m-%dT%H:%M:%S')
        # caso o cliente não tenha mais transações irá passar para o próximo
        if data_anterior == None:
            data_anterior = data
            id_anterior = id
            # print('inicial:', data)
        else:
            diff = data - data_anterior
            # se a diferença for menor do que 2 minutos 
            if diff < minute2:
                print(diff, id_anterior, id)
                fraudes.append((id_anterior, id, diff))
            data_anterior = data
            id_anterior = id
            # print(diff)
            # print('proxima:', data)

cliente: 718
cliente: 205
0:00:15 177 178
0:00:15 178 179
0:01:47 275 276
0:01:15 277 278
0:01:51 283 284
0:01:37 398 399
0:01:01 399 400
0:01:16 400 401
0:00:44 401 402
0:00:59 402 403
0:01:10 403 404
0:01:23 404 405
0:01:53 405 406
0:00:57 407 408
0:01:51 409 410
0:01:27 410 411
0:01:04 411 412
0:01:20 412 413
0:01:01 413 414
0:01:10 414 415
0:01:36 419 420
0:00:52 420 421
0:01:19 421 422
0:01:30 424 425
0:00:45 425 426
0:00:29 4084 4085
0:01:35 4085 4086
0:00:51 4149 4150
0:00:32 4411 4412
0:00:51 4412 4413
0:01:31 4413 4414
0:00:32 4432 4433
0:00:37 4446 4447
0:00:55 4447 4448
0:00:28 4455 4456
0:01:17 4466 4467
0:00:32 4467 4468
0:01:43 4468 4469
0:00:15 4469 4470
0:00:43 4470 4471
0:00:30 4472 4473
0:00:31 4474 4475
0:01:35 4477 4478
0:00:54 4483 4485
0:00:26 4485 4486
0:01:30 4490 4491
0:00:39 4504 4505
0:00:44 4505 4506
0:01:50 4514 4515
0:01:22 4528 4529
0:01:02 4529 4530
0:00:47 4530 4531
0:00:14 4531 4532
0:00:25 4541 4542
0:00:23 4542 4543
0:01:25 4551 4552
0:00:39 4557 455

In [174]:
# imprimindo a lista fraudes para verificar se está tudo certo
fraudes

[('177', '178', datetime.timedelta(seconds=15)),
 ('178', '179', datetime.timedelta(seconds=15)),
 ('275', '276', datetime.timedelta(seconds=107)),
 ('277', '278', datetime.timedelta(seconds=75)),
 ('283', '284', datetime.timedelta(seconds=111)),
 ('398', '399', datetime.timedelta(seconds=97)),
 ('399', '400', datetime.timedelta(seconds=61)),
 ('400', '401', datetime.timedelta(seconds=76)),
 ('401', '402', datetime.timedelta(seconds=44)),
 ('402', '403', datetime.timedelta(seconds=59)),
 ('403', '404', datetime.timedelta(seconds=70)),
 ('404', '405', datetime.timedelta(seconds=83)),
 ('405', '406', datetime.timedelta(seconds=113)),
 ('407', '408', datetime.timedelta(seconds=57)),
 ('409', '410', datetime.timedelta(seconds=111)),
 ('410', '411', datetime.timedelta(seconds=87)),
 ('411', '412', datetime.timedelta(seconds=64)),
 ('412', '413', datetime.timedelta(seconds=80)),
 ('413', '414', datetime.timedelta(seconds=61)),
 ('414', '415', datetime.timedelta(seconds=70)),
 ('419', '420', 

In [175]:
# cria uma lista vazia chamada 'id_t'
id_t = []

# itera sobre cada linha da lista 'fraudes'
for linha in fraudes:
    # adiciona o primeiro elemento da linha (linha[0]) à lista 'id_t'
    id_t.append(linha[0])
    # adiciona o segundo elemento da linha (linha[1]) à lista 'id_t'
    id_t.append(linha[1])

# retorna a lista 'id_t' com os valores adicionados
id_t

['177',
 '178',
 '178',
 '179',
 '275',
 '276',
 '277',
 '278',
 '283',
 '284',
 '398',
 '399',
 '399',
 '400',
 '400',
 '401',
 '401',
 '402',
 '402',
 '403',
 '403',
 '404',
 '404',
 '405',
 '405',
 '406',
 '407',
 '408',
 '409',
 '410',
 '410',
 '411',
 '411',
 '412',
 '412',
 '413',
 '413',
 '414',
 '414',
 '415',
 '419',
 '420',
 '420',
 '421',
 '421',
 '422',
 '424',
 '425',
 '425',
 '426',
 '4084',
 '4085',
 '4085',
 '4086',
 '4149',
 '4150',
 '4411',
 '4412',
 '4412',
 '4413',
 '4413',
 '4414',
 '4432',
 '4433',
 '4446',
 '4447',
 '4447',
 '4448',
 '4455',
 '4456',
 '4466',
 '4467',
 '4467',
 '4468',
 '4468',
 '4469',
 '4469',
 '4470',
 '4470',
 '4471',
 '4472',
 '4473',
 '4474',
 '4475',
 '4477',
 '4478',
 '4483',
 '4485',
 '4485',
 '4486',
 '4490',
 '4491',
 '4504',
 '4505',
 '4505',
 '4506',
 '4514',
 '4515',
 '4528',
 '4529',
 '4529',
 '4530',
 '4530',
 '4531',
 '4531',
 '4532',
 '4541',
 '4542',
 '4542',
 '4543',
 '4551',
 '4552',
 '4557',
 '4558',
 '4561',
 '4562',
 '4583

In [176]:
# converter o DataFrame do PySpark em um DataFrame do pandas
df_pandas = df_out.toPandas()

# selecionar as linhas do DataFrame do pandas onde o valor da coluna 'Id' está na lista 'id_t'
df_fraudes_out = df_pandas[df_pandas['Id'].isin(id_t)]

# exibir o DataFrame resultante contendo apenas as linhas onde o valor da coluna 'Id' está na lista 'id_t'
df_fraudes_out

,Id,Cliente_Id,Valor,DataHora,Hora,Minuto,Segundo,Dia
0,8607,910,-2.0,2022-01-19T20:15:26.000Z,20,15,26,2022-01-19
1,8608,910,-2.0,2022-01-19T20:14:56.000Z,20,14,56,2022-01-19
2,8609,910,-2.0,2022-01-19T20:14:26.000Z,20,14,26,2022-01-19
3,8610,910,-2.0,2022-01-19T20:13:56.000Z,20,13,56,2022-01-19
4,8612,910,-2.0,2022-01-19T20:13:26.000Z,20,13,26,2022-01-19
...,...,...,...,...,...,...,...,...
6356,33,74,-100.0,2019-09-05T15:11:11.000Z,15,11,11,2019-09-05
6357,32,74,-2.0,2019-09-05T15:09:45.000Z,15,9,45,2019-09-05
6358,31,74,-2.0,2019-09-05T15:09:30.000Z,15,9,30,2019-09-05
6359,30,74,-2.0,2019-09-05T15:09:07.000Z,15,9,7,2019-09-05


In [177]:
# salve o dataframe de fraudes out em um arquivo CSV
df_fraudes_out.to_csv('./dataset_fraudes_out.csv', index=False)

## Fraudes in 

In [178]:
# filtra as linhas com valor negativo na coluna "valor"
df_in = df.filter(col("Valor") >= 0)

# mostra o resultado
df_in.show()

+----+----------+-------+--------------------+----+------+-------+----------+
|  Id|Cliente_Id|  Valor|            DataHora|Hora|Minuto|Segundo|       Dia|
+----+----------+-------+--------------------+----+------+-------+----------+
|3120|       533|  5.001|2021-01-28T23:46:...|  23|    46|     47|2021-01-28|
|3119|       533| 24.999|2021-01-28T23:46:...|  23|    46|     47|2021-01-28|
|3108|       533|  5.001|2021-01-28T13:47:...|  13|    47|     37|2021-01-28|
|3107|       533|12.4995|2021-01-28T13:47:...|  13|    47|     36|2021-01-28|
|3106|       533|12.4995|2021-01-28T13:47:...|  13|    47|     36|2021-01-28|
|3092|       533| 24.999|2021-01-28T13:02:...|  13|     2|     57|2021-01-28|
|3091|       533|  5.001|2021-01-28T13:02:...|  13|     2|     57|2021-01-28|
|3079|       533| 24.999|2021-01-23T12:44:...|  12|    44|     31|2021-01-23|
|3078|       533|  5.001|2021-01-23T12:44:...|  12|    44|     30|2021-01-23|
|3069|       574|   20.0|2021-01-23T00:29:...|   0|    29|     5

In [179]:
# ordenar por Cliente_Id, agrupar por Cliente_Id e coleta os valores de DataHora
df_agrupado = df_in.orderBy(['Cliente_Id', 'DataHora']).groupBy("Cliente_Id") \
                .agg(collect_list("DataHora").alias("DataHora"), \
                     collect_list("Id").alias("Id"), \
                     collect_list("Valor").alias("Valor"))
                
# mostra o resultado
df_agrupado.show()

+----------+--------------------+--------------------+--------------------+
|Cliente_Id|            DataHora|                  Id|               Valor|
+----------+--------------------+--------------------+--------------------+
|       574|[2021-01-23T00:29...|              [3069]|              [20.0]|
|       718|[2021-07-28T23:19...|              [4801]|            [1000.0]|
|       205|[2021-06-02T21:45...|[3553, 3555, 3556...|[10.0, 20.0, 120....|
|       442|[2021-06-09T11:06...|              [3568]|               [5.0]|
|       727|[2021-08-28T16:55...|        [5028, 5037]|        [20.0, 30.0]|
|       495|[2020-10-31T13:55...|[1379, 1397, 1407...|[20.0, 16.808, 12...|
|       685|[2021-06-07T12:02...|[3563, 3580, 3582...|[5.0, 5.0, 5.0, 5.0]|
|       387|[2021-06-09T20:59...|[3571, 3576, 3623...|[20.0, 5.0, 20.0,...|
|       724|[2021-08-20T00:07...|              [4934]|              [20.0]|
|       534|[2020-12-19T02:13...|        [2690, 2833]|        [15.0, 10.0]|
|       586|

In [180]:
# itera sobre as linhas do DataFrame agrupado por cliente
lista = df_agrupado.collect()
for linha in lista:
    # recebe o valor do primeiro elemento da linha (cliente)
    cliente = linha[0]
    print(cliente)
    # itera simultaneamente sobre três listas (data, ID dos clientes e valor)
    for data, id, valor in zip(linha[1], linha[2], linha[3]):
        # imprime a data, o valor e o ID do cliente para cada elemento correspondente das três listas
        print(data, valor, id)

574
2021-01-23T00:29:52.000Z 20.0 3069
718
2021-07-28T23:19:32.000Z 1000.0 4801
205
2021-06-02T21:45:26.000Z 10.0 3553
2021-06-02T21:46:16.000Z 20.0 3555
2021-06-02T21:47:39.000Z 120.0 3556
2021-07-15T02:29:23.000Z 52.92 4464
2021-07-16T22:11:03.000Z 60.3 4503
2021-07-17T15:05:56.000Z 100.0 4527
2021-07-17T18:18:32.000Z 7.9365 4540
2021-07-17T18:34:30.000Z 48.0654 4544
2021-07-18T00:27:02.000Z 40.11 4554
2021-07-19T01:31:38.000Z 85.8 4581
2021-07-19T04:11:13.000Z 59.23125 4587
2021-07-21T23:59:10.000Z 100.0 4599
2021-07-23T02:39:52.000Z 80.256 4620
2021-07-23T22:10:54.000Z 47.6 4628
2021-07-23T22:10:54.000Z 26.4384 4627
2021-07-24T05:35:03.000Z 84.9 4643
2021-07-24T09:05:46.000Z 56.25 4647
2021-07-25T02:02:23.000Z 98.0 4664
2021-07-25T21:10:30.000Z 31.626125 4695
2021-07-28T13:35:31.000Z 44.0 4748
2021-07-28T13:35:31.000Z 29.34 4747
2021-07-28T20:16:18.000Z 82.865776 4793
2021-07-29T02:08:44.000Z 53.7824 4811
2021-07-29T02:08:44.000Z 60.75 4810
2021-07-29T02:28:39.000Z 27.89736 4812
20

In [181]:
# selecionando a coluna dataHora do dataset agrupado 
lista = df_agrupado.collect()

# definindo variável para 2 minutos
minute2 = timedelta(minutes=2)

# criando lista vazia chamada fraudes
fraudes = []

# estrutura de repetição para verificar em linha de cliente suas transações que possuem menos de dois minutos
# Percorre cada linha da lista
for linha in lista:
    # Extrai o primeiro elemento da linha, que é assumido como o identificador do cliente
    cliente = linha[0]
    # Imprime o cliente
    print('cliente:', cliente)
    # Compara a data da linha atual com a data da linha anterior e o ID do cliente da linha atual com o ID do cliente da linha anterior
    data_anterior = None
    id_anterior = None
    # verificamos cada dataHora da lista de transações do cliente e separamos as com diferença de menos de 2 min
    for data, id, valor in zip(linha[1], linha[2], linha[3]):
        data = datetime.strptime(data[:-5], '%Y-%m-%dT%H:%M:%S')
        # caso o cliente não tenha mais transações irá passar para o próximo
        if data_anterior == None:
            data_anterior = data
            id_anterior = id
            # print('inicial:', data)
        else:
            diff = data - data_anterior
            # se a diferença for menor do que 2 minutos 
            if diff < minute2:
                print(diff, id_anterior, id)
                fraudes.append((id_anterior, id, diff))
            data_anterior = data
            id_anterior = id
            # print(diff)
            # print('proxima:', data)

cliente: 574
cliente: 718
cliente: 205
0:00:50 3553 3555
0:01:23 3555 3556
0:00:00 4628 4627
0:00:00 4748 4747
0:00:00 4811 4810
cliente: 442
cliente: 727
cliente: 495
cliente: 685
cliente: 387
cliente: 724
cliente: 534
cliente: 586
0:01:20 8611 8613
0:01:01 8613 8615
cliente: 73
0:00:13 1937 1938
0:00:11 1938 1939
0:00:10 1939 1940
0:00:00 2133 2132
0:00:00 2152 2151
0:00:00 2199 2198
0:00:00 2198 2197
0:00:00 2259 2258
0:00:00 2292 2290
0:00:00 2390 2389
0:00:00 2389 2388
0:00:00 2421 2419
0:00:00 2419 2418
0:00:32 2579 2580
0:00:00 2612 2611
0:00:00 2649 2648
0:00:00 2661 2660
0:00:00 2660 2659
0:00:00 2749 2748
0:00:00 2967 2966
cliente: 443
cliente: 910
cliente: 389
cliente: 737
0:00:00 5314 5313
0:00:00 5372 5371
0:00:00 5371 5370
0:00:00 5387 5386
0:00:00 5714 5713
0:00:01 6380 6381
cliente: 695
0:00:53 3643 3644
cliente: 203
cliente: 570
cliente: 663
cliente: 730
0:00:33 5727 5728
0:00:40 5846 5847
0:00:09 5857 5858
0:01:37 5866 5868
0:00:22 6246 6247
0:00:30 6247 6248
0:00:18 

In [182]:
# cria uma lista vazia chamada 'id_t'
id_t = []

# itera sobre cada linha da lista 'fraudes'
for linha in fraudes:
    # adiciona o primeiro elemento da linha (linha[0]) à lista 'id_t'
    id_t.append(linha[0])
    # adiciona o segundo elemento da linha (linha[1]) à lista 'id_t'
    id_t.append(linha[1])

# retorna a lista 'id_t' com os valores adicionados
id_t

['3553',
 '3555',
 '3555',
 '3556',
 '4628',
 '4627',
 '4748',
 '4747',
 '4811',
 '4810',
 '8611',
 '8613',
 '8613',
 '8615',
 '1937',
 '1938',
 '1938',
 '1939',
 '1939',
 '1940',
 '2133',
 '2132',
 '2152',
 '2151',
 '2199',
 '2198',
 '2198',
 '2197',
 '2259',
 '2258',
 '2292',
 '2290',
 '2390',
 '2389',
 '2389',
 '2388',
 '2421',
 '2419',
 '2419',
 '2418',
 '2579',
 '2580',
 '2612',
 '2611',
 '2649',
 '2648',
 '2661',
 '2660',
 '2660',
 '2659',
 '2749',
 '2748',
 '2967',
 '2966',
 '5314',
 '5313',
 '5372',
 '5371',
 '5371',
 '5370',
 '5387',
 '5386',
 '5714',
 '5713',
 '6380',
 '6381',
 '3643',
 '3644',
 '5727',
 '5728',
 '5846',
 '5847',
 '5857',
 '5858',
 '5866',
 '5868',
 '6246',
 '6247',
 '6247',
 '6248',
 '6253',
 '6254',
 '6268',
 '6269',
 '6693',
 '6694',
 '7407',
 '7408',
 '2338',
 '2339',
 '2596',
 '2598',
 '2736',
 '2737',
 '3078',
 '3079',
 '3092',
 '3091',
 '3107',
 '3106',
 '3106',
 '3108',
 '3120',
 '3119',
 '3133',
 '3132',
 '3146',
 '3145',
 '3164',
 '3163',
 '3179',
 

In [184]:
# converter o DataFrame do PySpark em um DataFrame do pandas
df_pandas = df_in.toPandas()

# selecionar as linhas do DataFrame do pandas onde o valor da coluna 'Id' está na lista 'id_t'
df_fraudes_in = df_pandas[df_pandas['Id'].isin(id_t)]

# exibir o DataFrame resultante contendo apenas as linhas onde o valor da coluna 'Id' está na lista 'id_t'
df_fraudes_in

,Id,Cliente_Id,Valor,DataHora,Hora,Minuto,Segundo,Dia
0,3120,533,5.001,2021-01-28T23:46:47.000Z,23,46,47,2021-01-28
1,3119,533,24.999,2021-01-28T23:46:47.000Z,23,46,47,2021-01-28
2,3108,533,5.001,2021-01-28T13:47:37.000Z,13,47,37,2021-01-28
3,3107,533,12.4995,2021-01-28T13:47:36.000Z,13,47,36,2021-01-28
4,3106,533,12.4995,2021-01-28T13:47:36.000Z,13,47,36,2021-01-28
...,...,...,...,...,...,...,...,...
871,868,4,500.0,2020-08-27T01:21:13.000Z,1,21,13,2020-08-27
872,859,4,500.0,2020-08-27T00:55:58.000Z,0,55,58,2020-08-27
873,858,4,500.0,2020-08-27T00:55:40.000Z,0,55,40,2020-08-27
874,857,4,500.0,2020-08-27T00:55:17.000Z,0,55,17,2020-08-27


In [185]:
# salve o dataframe de fraudes in em um arquivo CSV
df_fraudes_in.to_csv('./dataset_fraudes_in.csv', index=False)

## Todas as fraudes

In [186]:
# leitura dos arquivos CSV
fraudes_in = pd.read_csv('dataset_fraudes_in.csv', sep=',')
fraudes_out = pd.read_csv('dataset_fraudes_out.csv', sep=',')

# concatenação horizontal dos arquivos
dataset_fraudes = pd.concat([fraudes_in, fraudes_out])
dataset_fraudes

,Id,Cliente_Id,Valor,DataHora,Hora,Minuto,Segundo,Dia
0,3120,533,5.0010,2021-01-28T23:46:47.000Z,23,46,47,2021-01-28
1,3119,533,24.9990,2021-01-28T23:46:47.000Z,23,46,47,2021-01-28
2,3108,533,5.0010,2021-01-28T13:47:37.000Z,13,47,37,2021-01-28
3,3107,533,12.4995,2021-01-28T13:47:36.000Z,13,47,36,2021-01-28
4,3106,533,12.4995,2021-01-28T13:47:36.000Z,13,47,36,2021-01-28
...,...,...,...,...,...,...,...,...
2554,33,74,-100.0000,2019-09-05T15:11:11.000Z,15,11,11,2019-09-05
2555,32,74,-2.0000,2019-09-05T15:09:45.000Z,15,9,45,2019-09-05
2556,31,74,-2.0000,2019-09-05T15:09:30.000Z,15,9,30,2019-09-05
2557,30,74,-2.0000,2019-09-05T15:09:07.000Z,15,9,7,2019-09-05


In [194]:
len(dataset_fraudes['Cliente_Id'].unique())

36

In [187]:
# salvando a junção dos csv em um arquivo final
dataset_fraudes.to_csv('dataset_fraudes.csv', index=False)